In [1]:
import json
import numpy as np
import cv2

In [2]:
# Read the main images

inputImage = cv2.imread('Trolley_problem.jpg')

# Read the templates

tramTemplate = cv2.imread('tram1.jpg')
humanTemplate = cv2.imread('h1.jpg')
humanSlantTemplate = cv2.imread('hr1.jpg')
leverTemplate = cv2.imread('lever.jpg')
leverFlipTemplate = cv2.flip(leverTemplate, 1)


In [3]:
# Store width and height of the templates in w and h

h1, w1, _ = tramTemplate.shape
h2, w2, _= humanTemplate.shape
h3, w3, _ = leverTemplate.shape
h4, w4, _ = humanSlantTemplate.shape
h5, w5, _ = leverFlipTemplate.shape

In [4]:
# Perform match operations.

tramResult = cv2.matchTemplate(inputImage, tramTemplate, cv2.TM_CCOEFF_NORMED)
humanResult = cv2.matchTemplate(inputImage, humanTemplate, cv2.TM_CCOEFF_NORMED)
leverResult = cv2.matchTemplate(inputImage, leverTemplate, cv2.TM_CCOEFF_NORMED)
humanSlantResult = cv2.matchTemplate(inputImage, humanSlantTemplate, cv2.TM_CCOEFF_NORMED)
leverFlipResult = cv2.matchTemplate(inputImage, leverFlipTemplate, cv2.TM_CCOEFF_NORMED)

In [5]:
threshold = 0.75 # Specify a threshold
font = cv2.FONT_HERSHEY_COMPLEX
object_detected = []
total_objects = 0
objects= ""

In [6]:
# Store the coordinates of matched area in a numpy array

loc_tram = np.where(tramResult >= threshold)
loc_human = np.where(humanResult >= threshold)
loc_lever = np.where(leverResult >= threshold)
loc_human_slant = np.where(humanSlantResult >= 0.80)
loc_lever_flip = np.where(leverFlipResult >= threshold)

In [7]:
# Line Detection

edges = cv2.Canny(inputImage, 150, 200, apertureSize=3)

minLineLength = 30  # Minimum length of line. Line segments shorter than this are rejected.
maxLineGap = 5  # Maximum allowed gap between line segments to treat them as single line
lines = cv2.HoughLinesP(image=edges,
                        rho=cv2.HOUGH_PROBABILISTIC,
                        theta=np.pi / 180,
                        threshold =30,
                        minLineLength = minLineLength,
                        maxLineGap = maxLineGap)

trackLoc = []

for x in range(0, len(lines)):
    for x1, y1, x2, y2 in lines[x]:
        pts = np.array([[x1, y1], [x2, y2]], dtype=np.int32)
        cv2.polylines(img=inputImage , pts=[pts], isClosed=True, color=(0, 255, 0))
        # trackLoc.append(pts.tolist())
        objects = "Tracks"

object_detected.append(objects)

In [8]:
# Draw a rectangle around the matched region.

# for pt in Zip(*loc[::-1]): this command is for the points which have values greater than threshold.
# zip is container of all such points, and it will iterate to all such points and draw rectangle around this closed entity

# detect tram

for tram_pt in zip(*loc_tram[::-1]):
    cv2.rectangle(inputImage, tram_pt, (tram_pt[0] + w1, tram_pt[1] + h1), (0, 255, 255), 1)
    # tramLoc = (tram_pt, (tram_pt[0] + w1, tram_pt[1] + h1))
    cv2.putText(inputImage, "Tram Detected", (200, 50), font, 0.5, 255)
    objects = "Tram"

object_detected.append(objects)

In [9]:
# detect humans

for human_pt in zip(*loc_human[::-1]):
    cv2.rectangle(inputImage, human_pt, (human_pt[0] + w2, human_pt[1] + h2), (0, 0, 255), 1)
    # humanLoc = (human_pt, (human_pt[0] + w2, human_pt[1] + h2))
    cv2.putText(inputImage, "Humans Detected", (800, 50), font, 0.5, 255)
    objects = "Human"

for human_slant_pt in zip(*loc_human_slant[::-1]):
    cv2.rectangle(inputImage, human_slant_pt, (human_slant_pt[0] + w4, human_slant_pt[1] + h4), (0, 0, 255), 1)

object_detected.append(objects)

In [10]:
# detect lever

for lever_pt in zip(*loc_lever[::-1]):
    cv2.rectangle(inputImage, lever_pt, (lever_pt[0] + w3, lever_pt[1] + h3), (255, 255, 0), 1)
    # leverLoc = (lever_pt, (lever_pt[0] + w3, lever_pt[1] + h3))
    cv2.putText(inputImage, "Lever Detected", (480, 50), font, 0.5, 255)
    objects = "Lever"

for lever_flip_pt in zip(*loc_lever_flip[::-1]):
    cv2.rectangle(inputImage, lever_flip_pt, (lever_flip_pt[0] + w5, lever_flip_pt[1] + h5), (255, 255, 0), 1)
    cv2.putText(inputImage, "Lever Detected", (480, 50), font, 0.5, 255)

object_detected.append(objects)

In [11]:
cv2.imwrite(r'outputImage.jpg', inputImage)

True

In [12]:
# write to json file

obj = []
for i, img_obj in enumerate(object_detected):
    obj.append((f'object {i}', img_obj))

data=dict(obj)
data['Total Objects in the image'] = len(data)

with open('result.json', 'w') as file:
    file.write(json.dumps(data, indent=4, separators=(',', ': ')))